# 调用GPT4接口获取是否变化

In [16]:
import os, json, re, ast, base64
from openai import OpenAI
import concurrent.futures
import threading
from time import sleep
# set the key
API_SECRET_KEY = "sk-zk22a10f1cb3d0807213c0138b58cc03ec666d7cb5d4c397"
BASE_URL = "https://flag.smarttrot.com/v1/"

In [17]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def write_to_file(file_path, content):
    with open(file_path, "w") as f:
        f.write(content)
        
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
result_path = os.path.join("gpt4-binary-results")
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [18]:
with open('LEVIR-MCI-dataset/LevirCCcaptions-v2-f.json', 'r') as f:
    datajson = json.load(f)

datajson = [x for x in datajson if x['filepath'] == 'test']

In [19]:
def chat_completions3(row):
    client = OpenAI(api_key=API_SECRET_KEY, base_url=BASE_URL)    
    prompt = (
        "Please determine whether the buildings and roads in these two remote sensing images have been newly built or removed. Only return yes or no, and do not return other content."
    )

    images = [f"LEVIR-MCI-dataset/images/test/A/{row['filename']}", f"LEVIR-MCI-dataset/images/test/B/{row['filename']}"]
    base64_images = [encode_image(image) for image in images]

    user_content = [{"type": "text", "text": prompt}]
    base64_images_content = [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "high",
            },
        }
        for base64_image in base64_images
    ]
    user_content.extend(base64_images_content)
    messages_template = [{"role": "user", "content": user_content}]

    # 构造请求并调用 API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages_template,
        max_tokens=1600,
        temperature=0,
        n=1,
        seed=2024,
    )

    # 打印生成结果
    result = response.choices[0].message.content
    return result

def get_result_binary(row):
    file_path = os.path.join(result_path, row['filename'].replace('.png','.txt'))
    if os.path.exists(file_path):
        return
    ans = chat_completions3(row)
    write_to_file(file_path, ans)

In [21]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 并行执行，提高效率
    executor.map(lambda x: get_result_binary(x), datajson)

## combine results

In [25]:
global_id = 100001
rslt = []
def get_ans(filename):
    with open(os.path.join("gpt4-binary-results", filename.replace(".png", ".txt")), "r") as f:
        return f.read()

for row in datajson:
    rslt.append({
        "question_id": global_id,
        "image_id": [f"test/A/{row['filename']}", f"test/B/{row['filename']}"],
        "answer": get_ans(row['filename'])
    })
    global_id += 1
with open('gpt4-binary-results.jsonl', 'w') as file:
    for item in rslt:
        file.write(json.dumps(item) + '\n')